# Redis Strings Tutorial

## Setup

In [11]:
%%capture
%pip install redis rich tqdm 

In [12]:
import redis
from rich.pretty import pprint

In [13]:
# creating cluster
# make sure jupyter server is connected to redis network
# `docker network connect redis_default jupyter-jupyter-1`
r = redis.RedisCluster(host='redis-master-1', port=6379)

## Redis Strings

Redis strings are used to store strings, numbers, images, serialized objects, and other kinds of information. They are the simplest of the Redis data types.

As a first example, use the `SET` command to store the value "Deimos", a type of bike, at key "bike:1".

In [14]:
# Create a new key
response = r.set("bike:1", "Deimos")
print(response)

True


You can retrieve the data using the `GET` command.

In [15]:
# Retrieve the data
value = r.get("bike:1")
print(value)

b'Deimos'


To see if a key exists, use the `EXISTS` command.

In [16]:
# Do these keys exist?
print(f"bike:1 exists: {r.exists('bike:1')}")
print(f"bike:2 exists: {r.exists('bike:2')}")

bike:1 exists: 1
bike:2 exists: 0


The `EXISTS` command returns `1` when a key exists and `0` when it does not exist. The `bike:1` key exists, so Redis returns a value of `1`. The `bike:2` key does not exist, so Redis returns `0`.

Use the `DEL` to delete a key and its data.

In [19]:
# Delete bike:1
r.delete("bike:1")
print(f"bike:1 exists after deletion: {r.exists('bike:1')}")

bike:1 exists after deletion: 0


## Strings as counters

If you store integer data in a string key, you can use these commands to increment and decrement its value:

- `INCR` - increment a number stored in a key by `1`.
- `INCRBY` - increment a number stored in a key by a specific positive or negative amount.
- `DECR` - decrement a number stored in a key by `1`.
- `DECRBY` - decrement a number stored in a key by a specific positive or negative amount.

In [20]:
# INCR usage
r.set("bikes:total_number", 10)
print(f"Initial value: {r.get('bikes:total_number')}")

# Increment twice
new_value1 = r.incr("bikes:total_number")
new_value2 = r.incr("bikes:total_number")
print(f"After incrementing twice: {r.get('bikes:total_number')}")

Initial value: b'10'
After incrementing twice: b'12'


In [21]:
# Use INCR to set a key to 1
r.delete("bikes:total_number")
new_value = r.incr("bikes:total_number")
print(f"After deleting and incrementing: {r.get('bikes:total_number')}")

After deleting and incrementing: b'1'


Note how the `INCR bikes:total_number` command that follows the `DEL` command re-creates the `bikes:total_number` key and sets its value to `1`.

In [22]:
# INCRBY usage
new_value = r.incrby("bikes:total_number", 100)
print(f"After incrementing by 100: {r.get('bikes:total_number')}")

After incrementing by 100: b'101'


In [23]:
# DECR and DECRBY usage
new_value1 = r.decr("bikes:total_number")
print(f"After decrementing: {r.get('bikes:total_number')}")

new_value2 = r.decrby("bikes:total_number", 10)
print(f"After decrementing by 10: {r.get('bikes:total_number')}")

After decrementing: b'100'
After decrementing by 10: b'90'


Using any of the `INCR*` or `DECR*` commands to modify the value of a key is an atomic operation. If two clients were to increment a key's value simultaneously, both operations would increment the key.

All the Redis operations implemented by single commands are atomic, including those that operate on more complex data structures. So, when you use a command that modifies the value of a key, any concurrent operations will also succeed.

## Key expiration

When keys are created without constraints, they live for the full lifespan of the Redis server to which they are attached. However, if you want a key to live for only a specific amount of time, you can use the `EXPIRE` command to alter its lifespan. To find out how much time is left before a key expires, use the `TTL` command.

In [24]:
# EXPIRE usage
r.set("bike:1:lock_status", "LOCKED")
r.expire("bike:1:lock_status", 20)
print(f"Key set with 20 seconds expiration")

Key set with 20 seconds expiration


After a few seconds, check the time-to-live status by running the `TTL` command.

In [30]:
# Check the bike:1:lock_status key
ttl = r.ttl("bike:1:lock_status")
print(f"Time to live: {ttl} seconds")

Time to live: -2 seconds


You'll see one of the two return values:

1. the number of seconds left before the key expires
2. `-2`, indicating that the key has already expired

If you try to use the `TTL` command on a key that does not have an expiration set, it will return `-1`.

In [31]:
# TTL on a non-expiring key
r.set("bike:2:lock_status", "LOCKED")
ttl = r.ttl("bike:2:lock_status")
print(f"TTL for non-expiring key: {ttl}")

TTL for non-expiring key: -1


`EXPIRE` and `TTL` operate in seconds, but there are analogous commands that operate in milliseconds. They are `PEXPIRE` and `PTTL`.

The `SET` command can take additional arguments, one of which allows you to set the value of a key and its time to live value directly in a single, atomic operation. The `EX` and `PX` arguments allow you to specify the TTL value as either seconds or milliseconds, respectively.

In [32]:
# SET with time to live
r.set("bike:1:lock_status", "LOCKED", px=1)
print("Key set with 1 millisecond expiration")

Key set with 1 millisecond expiration


It's possible to cancel a key's timeout using the `PERSIST` command.

In [33]:
# PERSIST usage
r.set("bike:1:lock_status", "LOCKED", ex=120)
print(f"TTL before persist: {r.ttl('bike:1:lock_status')} seconds")

r.persist("bike:1:lock_status")
print(f"TTL after persist: {r.ttl('bike:1:lock_status')}")

TTL before persist: 120 seconds
TTL after persist: -1


## Resources

- String type [reference page](https://redis.io/docs/data-types/strings?utm_source=redisinsight&utm_medium=main&utm_campaign=tutorials).
- Entire set of [Redis string commands](https://redis.io/commands/?group=string&utm_source=redisinsight&utm_medium=main&utm_campaign=tutorials).
